In [2]:
%pip install reportlab

Note: you may need to restart the kernel to use updated packages.


In [3]:
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
from reportlab.lib.units import cm
from reportlab.lib import colors
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.cidfonts import UnicodeCIDFont
from reportlab.pdfbase.ttfonts import TTFont


In [4]:
font_path = r"C:\Windows\Fonts\Arial.ttf"
pdfmetrics.registerFont(TTFont("Arial", font_path))

# registra a fonte chinesa embutida
pdfmetrics.registerFont(UnicodeCIDFont("STSong-Light"))

# --------- 5 exemplos de caracteres (hanzi, pinyin, tradução) ----------
CHARS = [
    ("水", "shuǐ", "água"),
    ("人", "rén", "pessoa"),
    ("大", "dà", "grande"),
    ("小", "xiǎo", "pequeno"),
    ("中", "zhōng", "meio"),
]

# --------- Parâmetros de layout ---------
PAGE_W, PAGE_H = A4

In [5]:
# Desenha o quadrado de treino em si
# parametros: 
    # c = folha 
    # x = coordenada horizontal do canto inferior esquerdo do retângulo
    # y = coordenada vertical do canto inferior esquerdo do retângulo
    # size = largura/ altura
    
def draw_mizige(c, x, y, size, dashed=True, diagonals=True):
    
    # Ativando o pontilhado na folha c
    if dashed:
        c.setDash(1, 2)  # traço 1, espaço 2

    # desenhando um quadrado 
    c.rect(x, y, size, size)

    # cruz
    c.line(x, y + size/2, x + size, y + size/2)
    c.line(x + size/2, y, x + size/2, y + size)

    # diagonais
    if diagonals:
        c.line(x, y, x + size, y + size)
        c.line(x, y + size, x + size, y)
    
    if dashed:
        c.setDash()  # volta ao traço contínuo
    

In [6]:
def draw_inner_char(c, cx, cy, hanzi, size_pt, mode="ghost"):
    """Caractere no centro do quadrado: ghost (cinza), dashed (contorno) ou none."""
    if mode == "none":
        return
    if mode == "ghost":
        c.saveState()
        c.setFont('STSong-Light', size_pt)
        c.setFillColorRGB(0.75, 0.75, 0.75)
        w = pdfmetrics.stringWidth(hanzi, 'STSong-Light', size_pt)
        c.drawString(cx - w/2, cy - size_pt*0.35, hanzi)
        c.restoreState()
    elif mode == "dashed":
        c.saveState()
        text = c.beginText()
        text.setFont('STSong-Light', size_pt)
        text.setTextRenderMode(1)    # contorno apenas
        c.setLineWidth(0.8)
        c.setDash(1, 2)
        w = pdfmetrics.stringWidth(hanzi, 'STSong-Light', size_pt)
        text.setTextOrigin(cx - w/2, cy - size_pt*0.35)
        text.textLine(hanzi)
        c.drawText(text)
        c.setDash()
        c.restoreState()

In [21]:
def write_chars(c, chars, start_x=40, start_y=None, line_gap=70):
    """
    Escreve pinyin + tradução em uma linha e o hanzi logo abaixo,
    iterando sobre a lista chars [(hanzi, pinyin, tradução), ...].
    - c: canvas
    - chars: lista de tuplas
    - start_x: margem esquerda
    - start_y: posição vertical inicial (se None = topo da página)
    - line_gap: espaço vertical entre blocos
    """
    if start_y is None:
        start_y = PAGE_H - 40  # começa do topo
    
    y = start_y
    for hanzi, pinyin, meaning in chars:
        # escreve pinyin + tradução
        c.setFont("Arial", 11)
        texto = f"{pinyin} - {meaning}"
        c.drawString(start_x, y, texto)
        
        # escreve o hanzi grande logo abaixo
        c.setFont("STSong-Light", 23)
        c.drawString(start_x, y - 30, hanzi)
        
        # desce para o próximo bloco
        y -= line_gap

In [ ]:
path=r"C:\Users\beatriz\Documents\chinese_codes\PDFs\teste.pdf"
c = canvas.Canvas(path, pagesize=A4)
s = 3 * cm  # 6 cm de lado
x = 40
y = PAGE_H - 10

write_chars(c, CHARS)

c.showPage()
c.save()
print(f"OK: {path}")

OK: C:\Users\beatriz\Documents\chinese_codes\PDFs\teste.pdf
